##Install Required Modules

In [0]:
# !pip install praw
# !pip install pandas
# !pip install nltk
# !pip install bs4
# !pip install re

##Import Libraires

In [0]:
import praw
import pandas as pd
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

##Praw variable declerations
PRAW is a Python wrapper for the Reddit API, which enables you to scrape data from subreddits. Here we define the subredit as India as we have to scrape data from that sub-reddit


In [0]:
reddit = praw.Reddit(client_id = "b5GZswE_-4JHvw",
                     client_secret = "weXodwTvLzJkPcnmfyP72DTs184",
                     user_agent = "Reddit Flare Detection",
                     username = "allergy21",
                     password = "##rkueCQf7ZGez!")

subreddit = reddit.subreddit('india')

##Flair categories
These are the flair categories (our labels) which we will classify

In [0]:
flairs = ["AskIndia", "Non-Political", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "Coronavirus"]


##Scarping data from reddit
Here we scrape 100 posts about each flair from the r/india subreddit and save it to pandas dataframe

In [0]:
topics_dict = {"flair":[], "title":[], "score":[], "id":[], "url":[], "comms_num": [], "created": [], "body":[], "author":[], "comments":[]}

for flair in flairs:
  print(flair)
  posts = subreddit.search(flair, limit=150)

  for post in posts:
    topics_dict["flair"].append(str(flair))
    topics_dict["title"].append(str(post.title))
    topics_dict["score"].append(str(post.score))
    topics_dict["id"].append(str(post.id))
    topics_dict["url"].append(str(post.url))
    topics_dict["comms_num"].append(str(post.num_comments))
    topics_dict["created"].append(str(post.created))
    topics_dict["body"].append(str(post.selftext))
    topics_dict["author"].append(str(post.author))

    # add comments
    post.comments.replace_more(limit=None)
    comment = ''
    count = 0
    for top_level_comment in post.comments:
      comment = comment + ' ' + top_level_comment.body
      count+=1     
      if(count > 10):
        break
    topics_dict["comments"].append(comment)

info = pd.DataFrame(topics_dict)


AskIndia
Non-Political
Scheduled
Photography
Science/Technology
Politics
Business/Finance
Policy/Economy
Sports
Food
Coronavirus


##Data Cleaning Functions
We define fuctions for data cleaning/pre-processing to remove unwanted symbols and stopwords

In [0]:
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
replace_symbol = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_by_space.sub(' ', text) # replace certain symbols by space in text
    text = replace_symbol.sub('', text) # delete symbols from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove STOPWORDS from text
    return text

def to_str(text):
  return str(text)

##Clean the data and save it into a CSV file

In [0]:
info['title'] = info['title'].apply(to_str)
info['body'] = info['body'].apply(to_str)
info['comments'] = info['comments'].apply(to_str)

info['title'] = info['title'].apply(clean_text)
info['body'] = info['body'].apply(clean_text)
info['comments'] = info['comments'].apply(clean_text)

combined_features = info["title"] + info["comments"] 
info = info.assign(combined_features = combined_features)

#saving the csv file
info.to_csv('data5.csv',index=False)